# 2.3 Class Definitions

## 2.3.1 Example: CreditCard Class

In [29]:
class CreditCard:

    def __init__(self, customer, bank, acnt, limit):
        self._customer = customer
        self._bank = bank
        self._account = acnt
        self._limit = limit
        self._balance = 0
        
    def get_customer(self):
        return self._customer
    
    def get_bank(self):
        return self._bank
    
    def get_account(self):
        return self._account
    
    def get_limit(self):
        return self._limit
    
    def get_balance(self):
        return self._balance
    
    def charge(self, price):
        if price + self._balance > self._limit:
            return False
        else:
            self._balance += price
            return True
        
    def make_payment(self, amount):
        self._balance -= amount

## 2.3.4 Iterators

In [1]:
class SequenceIterator:
    """An iterator for any of Python's sequence types."""
    
    def __init__(self, sequence):
        """Create an iterator for the given sequence."""
        self._seq = sequence    # _로 시작하는 내부 데이터 저장
        self._k = -1            # next를 처음 호출할 때 0이 되도록
        
    def __next__(self):
        """Return the next element, or else raise StopIteration error."""
        self._k += 1
        if self._k < len(self._seq):
            return self._seq[self._k]
        else:
            raise StopIteration()
            
    def __iter__(self):
        """By convention, an iterator must return itself as an iterator."""
        return self        

In [2]:
a = range(100)
seq_iter = SequenceIterator(a)

In [5]:
next(iter(seq_iter))

0

In [6]:
next(iter(seq_iter))

1

In [7]:
next(iter(seq_iter))

2

In [8]:
next(iter(seq_iter))

3

## 2.3.5 Example: Range Class

range 클래스를 구현해보자!

In [12]:
class Range:
    """A class that mimics the built-in range class."""
    
    def __init__(self, start, stop=None, step=1):
        """Initialize a Range instance.
        
        Semantics is similar to built-in range class.
        """
        if step == 0:
            raise ValueError('step cannot be 0')
        
        if stop is None:    # range(n) 같은 경우
            start, stop = 0, start    # range(0, n) 으로 변환
            
        # 길이 계산
        self._length = max(0, (stop - start + step - 1) // step)
        
        # start와 step을 알아야 __getitem__ 산출 가능
        self._start = start
        self._step = step
        
    def __len__(self):
        """레인지의 갯수 반환"""
        return self._length
    
    def __getitem__(self, k):
        """인덱스 k에 위치한 값을 반환(음수도 고려)"""
        if k<0:
            k += len(self)    # __len__ 활용
        
        if not 0 <= k < self._length:
            raise IndexError('index out of range')
            
        return self._start + k * self._step

In [20]:
r = Range(10)
len(r)

10

In [21]:
r = Range(3,10,3)
len(r)

3

In [22]:
for i in Range(len(r)):
    print(r[i])

3
6
9


In [23]:
r = Range(3,-10,-3)
len(r)

5

In [24]:
for i in Range(len(r)):
    print(r[i])

3
0
-3
-6
-9


In [25]:
r = Range(3,-10,-23)
len(r)

1

In [26]:
for i in Range(len(r)):
    print(r[i])

3


# 2.4 Inheritance

## 2.4.1 Extending the CreditCard Class

In [30]:
class PredatoryCreditCard(CreditCard):
    """An extension to CreditCard that compounds interest and fees."""
    
    def __init__(self, customer, bank, acnt, limit, apr):
        """Create a new predatory credit card instance.
        
        The initial balance is zero.
        
        apr      annual percentage rate (e.g., 0.0825 for 8.25% APR)
        """
        super().__init__(customer, bank, acnt, limit)
        self._apr = apr
        
    def charge(self, price):
        """ Charge given price to the card, assuming sufficient credit limit.
        
        Return True if charge was processed.
        Return False and assess $5 fee is charge is denied.
        """
        success = super().charge(price)    # 상속받은 메서드를 호출
        if not success:
            self._balance += 5
        return success
    
    def process_month(self):
        """Assess monthly interest on outstanding balance."""
        if self._balance > 0:
            # if pos. balance, convert APR to monthly mult. factor
            monthly_factor = pow(1+self._apr, 1/12)
            self._balance *= monthly_factor

## 2.4.2 Hierarchy of Numeric Progressions

Progression 클래스로부터 상속받는 수열 클래스를 만들자.

In [31]:
class Progression:
    """Iterator producing a generic progression.
    
    Default iterator produces the whole numbers 0, 1, 2,...
    """
    
    def __init__(self, start=0):
        """Initialize current to the first value of the progression."""
        self._current = start
        
    def _advance(self):
        """Update self._current to a new value.
        
        This should be overridden by a subclass to customize progression.
        
        By convention, if current is set to None, this designates the
        end of a finite progression.
        """
        self._current += 1
        
    def __next__(self):
        """Return the next element, or else raise StopIteration error."""
        if self._current is None:   # 이렇게 약속
            raise StopIteration()
        else:
            answer = self._current  # 리턴값 저장하고 하나 증가
            self._advance()
            return answer
        
    def __iter__(self):
        """By convention, an iterator must return itself as an iterator."""
        return self
    
    def print_progression(self, n):
        """Print next n values of the progression."""
        print(' '.join(str(next(self)) for j in range(n)))

In [33]:
seq = Progression()
seq.print_progression(5)

0 1 2 3 4


### Arithmetic Progression Class

In [34]:
class ArithmeticProgression(Progression):   # inherit from Progression
    """Iterator producing an arithmetic progression."""
    
    def __init__(self, increment=1, start=0):
        """Create a new arithmetric progression."""
        super().__init__(start)
        self._increment = increment
        
    def _advance(self):
        """Update current value by adding the fixed increment."""
        self._current += self._increment

In [37]:
seq = ArithmeticProgression(5,2)
seq.print_progression(5)

2 7 12 17 22


### Geometric Progression Class

In [38]:
class GeometricProgression(Progression):
    """Iterator producing a geometric progression."""
    
    def __init__(self, base=2, start=1):
        """Create a new geometric progression."""
        super().__init__(start)
        self._base = base
        
    def _advance(self):
        """Updating current value by multiplying it by the base value."""
        self._current *= self._base

In [39]:
seq = GeometricProgression(5,2)
seq.print_progression(5)

2 10 50 250 1250


### Fibonacci Progression Class

In [40]:
class FibonacciProgression(Progression):
    """Iterator producing a generalized Fibonacci progression."""
    
    def __init__(self, first=0, second=1):
        """Create a new fibonacci progression."""
        super().__init__(first)
        self._prev = second - first
        
    def _advance(self):
        """Update current value by taking sum of previous two."""
        self._prev, self._current = self._current, self._prev + self._current

In [42]:
seq = FibonacciProgression()
seq.print_progression(20)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181


## 2.4.3 Abstract Base Classes

ABC 클래스는 추상적으로 정의되는 클래스로서 인스턴스를 만드는 것은 불가능하다. 다만 이 클래스를 상속받는 클래스들이 갖춰야할 구조를 선언하는 역할을 하며, 특히 `@abstractmethod` 선언을 한 메서드들은 상속받는 클래스들이 반드시 정의해줘야 한다.

# 2.5 Namespaces and Object-Orientation

대충 뭔 말인지는 알겠는데 왜 중요한 지는 잘 모르겠음...

# 2.6 Shallow and Deep Copying

클래스의 인스턴스들을 담고 있는 리스트 A 가 있을 때, 이를 복사하는 방법은 다음 세 가지가 있다.

1. B = A : 동일한 대상을 참조하므로 A를 수정하면 B도 변경됨

2. B = list(A) : 얕은 복사를 수행하므로 리스트 A에 포함된 멤버를 추가하거나 삭제해도 B가 변경되진 않지만, A의 멤버 중 하나의 인스턴스를 수정하면 이를 참조하고 있는 B의 인스턴스도 변경됨

3. B = copy.deepcopy(A) : 리스트의 멤버까지 모두 새롭게 복사하여 생성